# Amazon Bedrock boto3 Setup

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. You can also run on a local setup, as long as you have the right IAM credentials to invoke the Claude model via Bedrock*

---

In this demo notebook, we demonstrate an implementation of Function Calling with Anthropic's Claude models via Bedrock. This notebook is inspired by the [original work](https://drive.google.com/drive/folders/1-94Fa3HxEMkxkwKppe8lp_9-IXXvsvv1) by the Anthropic Team and modified it for use with Amazon Bedrock.

---

## Prerequisites

Run the cells in this section to install the needed packages for workshop

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57" \
    "requests" \
    "defusedxml"

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [18]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Anthropic Claude

The crux of this example is to let Claude models know about a set of `tools` that it has available i.e. functions it can call between a set of tags. This is possible because Anthropic's Claude models have been extensively trainied on such tags in its training corpus.

Then present a way to call the tools in a step by step fashion till it gets the right answer. We create a set of callable functions in another file called `tools.py`

A sample `tools.py` is added to the same folder of this notebook and can be modified to suit your needs. Once you have modified `tools.py`, import it so that we have access to it in this notebook

In [6]:
import tools

We then create a set of auxillary functions that help create the input prompt.

In [7]:
def create_prompt(tools_string, user_input):
    prompt_template = f"""
In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

Human:
{user_input}


Assistant:
"""
    return prompt_template

In [8]:
def add_tools():
    tools_string = ""
    for tool_spec in tools.list_of_tools_specs:
        tools_string += tool_spec
    return tools_string

In [11]:
from typing import Any
from defusedxml import ElementTree
from collections import defaultdict
# print(add_tools())
# Uncomment print to test if tools is being imported correctly and your functions are correctly being interpreted via the tags.

This `call_function` will be used later to extract the name of the tool from your `tools.py` file and call it from the output of Claude. A few more helper functions are defined and can be used as is without modification for your use case.

In [12]:
def call_function(tool_name, parameters):
    func = getattr(tools, tool_name)
    output = func(**parameters)
    return output

In [14]:
def format_result(tool_name, output):
    return f"""
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{output}
</stdout>
</result>
</function_results>
"""

In [15]:
def etree_to_dict(t) -> dict[str, Any]:
    d = {t.tag: {}}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(("@" + k, v) for k, v in t.attrib.items())
    if t.text and t.text.strip():
        if children or t.attrib:
            d[t.tag]["#text"] = t.text
        else:
            d[t.tag] = t.text
    return d

Here is where we can glue all the pieces together. Print the final prompt data to double check if the input is you expect.

In [16]:
user_input = "Can you check the weather for me in San Francisco?"
tools_string = add_tools()
prompt_data = create_prompt(tools_string, user_input)
print(prompt_data)


In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

<tool_description>
<tool_name>get_weather</tool_name>
<description>
Returns weather data for a given latitude and longitude. </description>
<parameters>
<parameter>
<name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description>
</parameter>
</parameters>
</tool_description>
<tool_description>
<tool_name>get_lat_long</tool_name>
<description>
Returns the latitude and longitude for a given pl

This next cell is to test the response of the Claude models based on your constructed input. Note that we have not instrumented output to call the actual functions, but this should give you an idea on how Claude's output can be parsed and the corresponding functions can be subsequently called.

In [19]:
body = json.dumps({"prompt": prompt_data, 
                   "max_tokens_to_sample": 1000,
                   "temperature": 0,
                   "stop_sequences": ["\n\nHuman:"]})
# modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

 Here is how I can check the weather in San Francisco:

<function_calls>
<invoke>
<tool_name>get_lat_long</tool_name>  
<parameters>
<place>San Francisco</place>
</parameters>
</invoke>
</function_calls>

This will first get the latitude and longitude coordinates for San Francisco. 

Then I can pass those coordinates to the get_weather tool:

<function_calls>
<invoke>
<tool_name>get_weather</tool_name>
<parameters>
<latitude>37.7749</latitude>
<longitude>-122.4194</longitude>  
</parameters>
</invoke> 
</function_calls>

This will retrieve the current weather data for those coordinates, giving us the weather in San Francisco.

Let me know if you need me to actually call the tools and get the weather report!


### Run loop

This function is the actual orchestrator of the function calling logic. Here's how it works:

1. We kick off a loop that first calls Claude with our tool use prompt with the tool specs and the user input loaded into it.
2. We get the completion from Claude and check if the stop sequence for the completion was the closing tag for a function call, ```</function_calls>```
3. If the completion does in fact contain a function call, we extract out the tool name and the tool parameters from the tags.
4. We then call the function that Claude has decided to invoke using our helped auxillary function.
5. We take the results of the function call, format them into an tag structure, and add them back to the prompt. This works because with subsequent calls, we are basically pre-filling the output of the model and asking it to pick up where it left off, with addition data from the previous results.
6. We repeat the loop starting at step 1 with the original prompt plus the text that has been appended.
7. This process continues until Claude finally outputs an answer and we break the loop.

In [20]:
def run_loop(prompt):
    print(prompt)
    # Start function calling loop
    while True:
        body = json.dumps({"prompt": prompt, 
                   "max_tokens_to_sample": 1000,
                   "temperature": 0,
                   "stop_sequences": ["\n\nHuman:", "</function_calls>"]})
        # modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
        modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
        accept = "application/json"
        contentType = "application/json"

        # Get a completion from Claude
        try:
            response = bedrock_runtime.invoke_model(
                body=body, modelId=modelId, accept=accept, contentType=contentType
            )
            response_body = json.loads(response.get("body").read())
            partial_completion = response_body.get("completion")
            stop_reason = response_body.get("stop_reason") 
        except botocore.exceptions.ClientError as error:
            if error.response['Error']['Code'] == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\
                        \nTo troubeshoot this issue please refer to the following resources.\
                        \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                        \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
            else:
                raise error

        # Append the completion to the end of the prommpt
        prompt += partial_completion
        if stop_reason == 'stop_sequence':
            # If Claude made a function call
            print(partial_completion)
            start_index = partial_completion.find("<function_calls>")

            if start_index != -1:
                # Extract the XML Claude outputted (invoking the function)
                extracted_text = partial_completion[start_index+16:]

                # Parse the XML find the tool name and the parameters that we need to pass to the tool
                xml = ElementTree.fromstring(extracted_text)
                tool_name_element = xml.find("tool_name")
                if tool_name_element is None:
                    print("Unable to parse function call, invalid XML or missing 'tool_name' tag")
                    break
                tool_name_from_xml = tool_name_element.text.strip()
                parameters_xml = xml.find("parameters")
                if parameters_xml is None:
                    print("Unable to parse function call, invalid XML or missing 'parameters' tag")
                    break
                param_dict = etree_to_dict(parameters_xml)
                parameters = param_dict["parameters"]

                # Call the tool we defined in tools.py
                output = call_function(tool_name_from_xml, parameters)

                # Add the stop sequence back to the prompt
                prompt += "</function_calls>"
                print("</function_calls> -- appending")

                # Add the result from calling the tool back to the prompt
                function_result = format_result(tool_name_from_xml, output)
                print(function_result)
                prompt += function_result
            else: # Once the output of the prompt does not have the <function_calls>, the start_index will be = -1, which means no more functions to be called. 
                break
        else:
            # If Claude did not make a function call
            # outputted answer
            print(partial_completion)
            break

Let's run it all together now.

In [21]:
run_loop(prompt_data)


In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

<tool_description>
<tool_name>get_weather</tool_name>
<description>
Returns weather data for a given latitude and longitude. </description>
<parameters>
<parameter>
<name>latitude</name>
<type>string</type>
<description>The latitude coordinate as a string</description>
</parameter> <parameter>
<name>longitude</name>
<type>string</type>
<description>The longitude coordinate as a string</description>
</parameter>
</parameters>
</tool_description>
<tool_description>
<tool_name>get_lat_long</tool_name>
<description>
Returns the latitude and longitude for a given pl

## Next steps

In this notebook we showed some basic examples of invoking Amazon Bedrock models using the AWS Python SDK. You're now ready to explore the other labs to dive deeper on different use-cases and patterns.